In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

INPUT_FILE = "Uitgebreide_VKM_dataset.csv"
OUTPUT_FILE = "Uitgebreide_VKM_dataset_cleaned.csv"

# Kolommen die we willen behouden in de finale dataset
KEEP_COLUMNS = [
    "id", "name", "shortdescription", "description", "content",
    "studycredit", "location", "contact_id", "level", "learningoutcomes",
    "module_tags", "interests_match_score", "popularity_score",
    "estimated_difficulty", "available_spots", "start_date"
]

# Kolommen die we als tekst willen schoonmaken
TEXT_COLS = [
    "shortdescription",
    "description",
    "content",
    "learningoutcomes"
]

# Stopwoorden en lemmatizer
# Kies 'english' of 'dutch' afhankelijk van je dataset
STOP_LANG = "dutch"   # of "english"
stop_words = set(stopwords.words(STOP_LANG))
lemmatizer = WordNetLemmatizer()

def normalize_length(tokens, max_len=200):
    """Beperk tokens tot een vaste maximale lengte om scheefheid te verminderen."""
    return tokens[:max_len]


def normalize_text(text: str) -> str:
    """Maak tekst schoon, lemmatiseer en normaliseer lengte."""
    if pd.isna(text):
        return ""

    text = str(text).lower()

    # Verwijder alles behalve letters, cijfers en spaties
    text = re.sub(r"[^a-zA-Z0-9\sáéíóúàèìòùäëïöüâêîôûçñ]", " ", text)

    tokens = word_tokenize(text)

    # Filter op stopwoorden en korte tokens
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    # Lemmatiseer
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    tokens = normalize_length(tokens, max_len=200)

    return " ".join(tokens)


def fill_missing_content(df: pd.DataFrame) -> pd.DataFrame:
    """Vul lege shortdescription/description/content met intelligente fallbacks."""
    
    # 1. shortdescription: gebruik description[:200] of name
    mask = df['shortdescription'].isna() | (df['shortdescription'].str.strip() == '') | (df['shortdescription'].str.lower() == 'ntb')
    df.loc[mask, 'shortdescription'] = df.loc[mask].apply(
        lambda row: row['description'][:200] if pd.notna(row['description']) and row['description'].strip() and row['description'].lower() != 'ntb'
        else f"{row['name']} - Een verdiepende module voor verdere professionele ontwikkeling.",
        axis=1
    )
    
    # 2. description: gebruik content of shortdescription + name
    mask = df['description'].isna() | (df['description'].str.strip() == '') | (df['description'].str.lower() == 'ntb')
    df.loc[mask, 'description'] = df.loc[mask].apply(
        lambda row: row['content'] if pd.notna(row['content']) and row['content'].strip() and row['content'].lower() != 'ntb'
        else f"In de module {row['name']} verdiep je je in {row['shortdescription'] if pd.notna(row['shortdescription']) else 'relevante onderwerpen'}. Deze module biedt praktijkgerichte kennis en vaardigheden die je voorbereidt op professioneel werken in het vakgebied.",
        axis=1
    )
    
    # 3. content: gebruik description
    mask = df['content'].isna() | (df['content'].str.strip() == '') | (df['content'].str.lower() == 'ntb')
    df.loc[mask, 'content'] = df.loc[mask, 'description']
    
    return df


def fill_learning_outcomes(df: pd.DataFrame) -> pd.DataFrame:
    """Vul learningoutcomes met passende waarden op basis van context."""
    
    # Generieke learning outcomes per niveau
    nlqf5_default = "Na afronding van deze module beschik je over de kennis en vaardigheden om professioneel te handelen binnen dit vakgebied. Je kunt theoretische kennis toepassen in praktijksituaties en reflecteert op je eigen handelen."
    
    nlqf6_default = "Na afronding van deze module kun je complexe vraagstukken analyseren en oplossen binnen dit vakgebied. Je werkt methodisch, reflecteert kritisch op je handelen en draagt bij aan innovatie en kennisontwikkeling in de professionele praktijk."
    
    # Specifieke learning outcomes op basis van module_tags
    tag_based_outcomes = {
        'technologie': "Je past technologische innovaties toe in de praktijk en evalueert hun impact op het vakgebied.",
        'zorg': "Je levert zorg op professioneel niveau, afgestemd op de behoeften van de zorgvrager en in lijn met beroepscodes.",
        'welzijn': "Je draagt bij aan het welzijn van individuen en groepen door passende interventies en ondersteuning te bieden.",
        'ondernemen': "Je ontwikkelt ondernemende vaardigheden en kunt zakelijke kansen identificeren en realiseren.",
        'internationaal': "Je werkt in internationale context en toont interculturele competenties.",
        'jeugd': "Je werkt methodisch met jeugdigen en hun systemen, rekening houdend met ontwikkelingspsychologische aspecten.",
        'ontwerp': "Je ontwikkelt en evalueert ontwerpen die voldoen aan gebruikerseisen en professionele standaarden.",
        'onderzoek': "Je voert praktijkgericht onderzoek uit en vertaalt bevindingen naar concrete aanbevelingen.",
    }
    
    def generate_learning_outcome(row):
        if pd.notna(row['learningoutcomes']) and row['learningoutcomes'].strip() and row['learningoutcomes'].lower() not in ['ntb', 'nader te bepalen', 'nog niet bepaald']:
            return row['learningoutcomes']
        
        # Basis outcome op basis van niveau
        level = row.get('level', 'NLQF5')
        base_outcome = nlqf6_default if 'NLQF6' in str(level) else nlqf5_default
        
        # Probeer tag-based outcome toe te voegen
        tags = row.get('module_tags', [])
        if isinstance(tags, str):
            tags = tags.lower().split(',')
        elif isinstance(tags, list):
            tags = [str(t).lower() for t in tags]
        
        specific_outcome = ""
        for tag in tags:
            tag_clean = tag.strip().strip("[]'\"")
            for key, outcome in tag_based_outcomes.items():
                if key in tag_clean:
                    specific_outcome = f" {outcome}"
                    break
            if specific_outcome:
                break
        
        return base_outcome + specific_outcome
    
    df['learningoutcomes'] = df.apply(generate_learning_outcome, axis=1)
    
    return df


def parse_module_tags(tag_value):
    """Parse module_tags van string naar lijst."""
    import ast
    
    if pd.isna(tag_value):
        return []
    
    # Als het al een lijst is
    if isinstance(tag_value, list):
        return tag_value
    
    # Converteer naar string en clean
    tag_str = str(tag_value).strip()
    
    # Lege string
    if not tag_str or tag_str == '':
        return []
    
    # Check voor 'ntb' varianten
    if tag_str.lower() in ['ntb', '[]', "['ntb']", '["ntb"]']:
        return []
    
    try:
        # Probeer als Python literal te evalueren
        parsed = ast.literal_eval(tag_str)
        if isinstance(parsed, list):
            # Filter 'ntb' tags eruit
            return [tag for tag in parsed if str(tag).lower() != 'ntb']
        return []
    except (ValueError, SyntaxError):
        # Als het geen geldige Python literal is, split op komma's
        if ',' in tag_str:
            tags = [t.strip().strip("[]'\"") for t in tag_str.split(',')]
            return [tag for tag in tags if tag and tag.lower() != 'ntb']
        else:
            # Enkele tag
            cleaned = tag_str.strip("[]'\"")
            if cleaned and cleaned.lower() != 'ntb':
                return [cleaned]
            return []


def generate_tags_from_text(row):
    """Genereer automatisch tags op basis van module naam en beschrijving."""
    
    # Verzamel tekst voor analyse
    text_parts = []
    if pd.notna(row.get('name')):
        text_parts.append(str(row['name']).lower())
    if pd.notna(row.get('shortdescription')):
        text_parts.append(str(row['shortdescription']).lower())
    
    combined_text = ' '.join(text_parts)
    
    # Woordenlijst voor tag detectie (meest relevante keywords)
    tag_keywords = {
        'technologie': ['technologie', 'digitalisering', 'digitaal', 'ict', 'software', 'data', 'ai', 'robotica', 'bim', 'virtual reality'],
        'zorg': ['zorg', 'verpleegkunde', 'verpleegkundige', 'patient', 'gezondheidszorg', 'medisch', 'behandeling', 'therapie'],
        'welzijn': ['welzijn', 'welzijnswerk', 'maatschappelijk', 'sociaal werk', 'participatie', 'inclusie'],
        'psychologie': ['psychologie', 'psychisch', 'gedrag', 'mentaal', 'cognitief', 'emotioneel'],
        'onderwijs': ['onderwijs', 'pedagogisch', 'didactisch', 'lesgeven', 'docent', 'leren', 'onderwijzen'],
        'management': ['management', 'organisatie', 'leiderschap', 'projectmanagement', 'bedrijfskunde'],
        'innovatie': ['innovatie', 'innovatief', 'vernieuwing', 'ontwikkeling', 'design thinking'],
        'internationaal': ['internationaal', 'buitenland', 'intercultureel', 'global', 'abroad'],
        'jeugd': ['jeugd', 'kinderen', 'jongeren', 'kind', 'baby', 'adolescent'],
        'ontwerp': ['ontwerp', 'design', 'creatie', 'vormgeving', 'architectuur', 'bouwen'],
        'onderzoek': ['onderzoek', 'research', 'analyse', 'wetenschappelijk'],
        'duurzaamheid': ['duurzaam', 'duurzaamheid', 'circulair', 'groen', 'klimaat', 'milieu', 'biobased'],
        'ondernemen': ['ondernemen', 'ondernemerschap', 'business', 'startup', 'zzp'],
        'communicatie': ['communicatie', 'gesprek', 'presenteren', 'storytelling'],
        'gezondheid': ['gezondheid', 'preventie', 'leefstijl', 'beweging', 'voeding'],
    }
    
    # Detecteer relevante tags
    detected_tags = []
    for tag, keywords in tag_keywords.items():
        for keyword in keywords:
            if keyword in combined_text:
                detected_tags.append(tag)
                break  # Één match per tag is genoeg
    
    # Als er geen tags gedetecteerd zijn, maak generieke tags op basis van woorden
    if not detected_tags:
        # Haal belangrijke woorden uit de naam
        words = str(row.get('name', '')).lower().split()
        # Filter stopwoorden en korte woorden
        important_words = [w for w in words if len(w) > 4 and w not in ['deze', 'voor', 'binnen', 'module', 'minor']]
        detected_tags = important_words[:3]  # Max 3 woorden als fallback
    
    return detected_tags[:5]  # Max 5 tags per module


def replace_ntb_values(df: pd.DataFrame) -> pd.DataFrame:
    """Vervang alle 'ntb' of 'Ntb' waarden met 'Nog niet bepaald'."""
    
    # Definieer kolommen waar we ntb willen vervangen (tekstuele kolommen)
    text_columns = ['shortdescription', 'description', 'content', 'learningoutcomes', 'name']
    
    for col in text_columns:
        if col in df.columns:
            # Vervang ntb/Ntb/NTB met "Nog niet bepaald"
            mask = df[col].str.lower().str.strip() == 'ntb'
            df.loc[mask, col] = 'Nog niet bepaald'
    
    # Voor module_tags: parse van string naar lijst
    if 'module_tags' in df.columns:
        df['module_tags'] = df['module_tags'].apply(parse_module_tags)
    
    return df


def fill_empty_tags(df: pd.DataFrame) -> pd.DataFrame:
    """Vul lege module_tags met automatisch gegenereerde tags."""
    
    if 'module_tags' not in df.columns:
        return df
    
    # Vind modules zonder tags
    empty_mask = df['module_tags'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True)
    
    # Genereer tags voor modules zonder tags
    df.loc[empty_mask, 'module_tags'] = df[empty_mask].apply(generate_tags_from_text, axis=1)
    
    return df



def clean_vkm_dataset(input_file: str, output_file: str) -> pd.DataFrame:
    # Data inladen
    df = pd.read_csv(input_file)

    print("=" * 60)
    print("DATA CLEANING & ENRICHMENT PROCESS")
    print("=" * 60)
    print(f"Origineel: {df.shape[0]} rijen, {df.shape[1]} kolommen")

    df_cleaned = df.copy()

    # 1. Selecteer alleen de gewenste kolommen
    available_cols = [c for c in KEEP_COLUMNS if c in df_cleaned.columns]
    missing_cols = [c for c in KEEP_COLUMNS if c not in df_cleaned.columns]
    
    if missing_cols:
        print(f"\nVolgende kolommen ontbreken: {missing_cols}")
    
    df_cleaned = df_cleaned[available_cols]
    print(f"\n1. Kolommen geselecteerd: {len(available_cols)} van {len(KEEP_COLUMNS)}")
    print(f"   Behouden: {available_cols}")

    # 2. Vervang alle 'ntb' waarden met "Nog niet bepaald"
    print("\n2. Vervangen van 'ntb' waarden")
    before_counts = {}
    for col in ['shortdescription', 'description', 'content', 'learningoutcomes']:
        if col in df_cleaned.columns:
            before_counts[col] = df_cleaned[col].str.lower().str.strip().eq('ntb').sum()
    
    df_cleaned = replace_ntb_values(df_cleaned)
    
    for col, count in before_counts.items():
        print(f"   {col}: {count} 'ntb' waarden vervangen")
    
    # 2b. Vul lege module_tags met auto-gegenereerde tags
    print("\n2b. Auto-genereren van module tags voor modules zonder tags")
    before_empty = df_cleaned['module_tags'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True).sum()
    
    df_cleaned = fill_empty_tags(df_cleaned)
    
    after_empty = df_cleaned['module_tags'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True).sum()
    print(f"   {before_empty} modules zonder tags → {after_empty} na auto-generatie")

    # 3. Vul lege waarden in content velden
    print("\n3. Vullen van lege content velden")
    before_nulls = {
        'shortdescription': df_cleaned['shortdescription'].isna().sum() + (df_cleaned['shortdescription'] == '').sum(),
        'description': df_cleaned['description'].isna().sum() + (df_cleaned['description'] == '').sum(),
        'content': df_cleaned['content'].isna().sum() + (df_cleaned['content'] == '').sum()
    }
    
    df_cleaned = fill_missing_content(df_cleaned)
    
    for col, count in before_nulls.items():
        after = df_cleaned[col].isna().sum() + (df_cleaned[col] == '').sum()
        print(f"   {col}: {count} lege velden → {after} na vullen")

    # 4. Vul learning outcomes met passende waarden
    print("\n4. Genereren van learning outcomes")
    before_lo = df_cleaned['learningoutcomes'].apply(
        lambda x: pd.isna(x) or str(x).strip() == '' or str(x).lower() in ['ntb', 'nader te bepalen', 'nog niet bepaald']
    ).sum()
    
    df_cleaned = fill_learning_outcomes(df_cleaned)
    
    after_lo = df_cleaned['learningoutcomes'].apply(
        lambda x: pd.isna(x) or str(x).strip() == '' or str(x).lower() in ['ntb', 'nader te bepalen']
    ).sum()
    print(f"   {before_lo} ontbrekende learning outcomes → {after_lo} na vullen")

    # 5. start_date naar geldige datetime
    if "start_date" in df_cleaned.columns:
        df_cleaned["start_date"] = pd.to_datetime(df_cleaned["start_date"], errors="coerce")
        invalid_dates = df_cleaned["start_date"].isna().sum()
        print(f"\n5. start_date geconverteerd naar datetime")
        print(f"   Ongeldige datums naar NaT: {invalid_dates}")

    # 6. Duplicaten op id droppen
    if "id" in df_cleaned.columns:
        before = df_cleaned.shape[0]
        duplicates = df_cleaned.duplicated(subset=["id"]).sum()
        df_cleaned = df_cleaned.drop_duplicates(subset=["id"])
        after = df_cleaned.shape[0]
        print(f"\n6. Duplicaten op 'id'")
        print(f"   Gevonden: {duplicates}, Rijen voor: {before}, na: {after}")

    # 7. Tekstvelden schoonmaken + lemmatizeren
    print("\n7. Tekstvelden normaliseren en lemmatiseren")
    for col in TEXT_COLS:
        if col in df_cleaned.columns:
            clean_col = f"{col}_clean"
            print(f"   - Verwerken: {col} -> {clean_col}")
            df_cleaned[clean_col] = df_cleaned[col].apply(normalize_text)

    # 8. Globale missing value check
    total_nulls = df_cleaned.isnull().sum().sum()
    print("\n" + "=" * 60)
    print("FINALE DATASET STATUS")
    print("=" * 60)
    print(f"Rijen: {df_cleaned.shape[0]}")
    print(f"Kolommen: {df_cleaned.shape[1]}")
    print(f"Totaal NULL waarden: {total_nulls}")

    # Optioneel: per kolom
    print("\nNULL waarden per kolom (alleen > 0):")
    nulls_per_col = df_cleaned.isnull().sum()
    nulls_display = nulls_per_col[nulls_per_col > 0]
    if len(nulls_display) > 0:
        print(nulls_display)
    else:
        print("  Geen NULL waarden gevonden! ")

    # Data quality check
    print("\n" + "=" * 60)
    print("DATA QUALITY VERIFICATIE")
    print("=" * 60)
    
    # Check voor 'ntb' waarden die nog over zijn
    ntb_remaining = 0
    for col in ['shortdescription', 'description', 'content', 'learningoutcomes']:
        if col in df_cleaned.columns:
            count = df_cleaned[col].str.lower().str.strip().eq('ntb').sum()
            ntb_remaining += count
            if count > 0:
                print(f"{col}: {count} 'ntb' waarden nog aanwezig!")
    
    if ntb_remaining == 0:
        print("Geen 'ntb' waarden meer aanwezig")
    
    # Check voor lege velden
    empty_critical = 0
    for col in ['shortdescription', 'description', 'content']:
        if col in df_cleaned.columns:
            count = (df_cleaned[col].isna() | (df_cleaned[col].str.strip() == '')).sum()
            empty_critical += count
            if count > 0:
                print(f"{col}: {count} lege velden")
    
    if empty_critical == 0:
        print("Alle kritieke tekstvelden zijn gevuld")
    
    # Check module_tags
    if 'module_tags' in df_cleaned.columns:
        empty_tags = df_cleaned['module_tags'].apply(lambda x: len(x) == 0 if isinstance(x, list) else True).sum()
        total_tags = len(df_cleaned)
        print(f"\nModule tags: {total_tags - empty_tags}/{total_tags} modules hebben tags")
        if empty_tags > 0:
            print(f"   {empty_tags} modules zonder tags (dit kan normaal zijn)")

    # Sample tonen
    sample_cols = [c for c in ["id", "name", "shortdescription", "learningoutcomes"] if c in df_cleaned.columns]
    print("\nSample (eerste 3 rijen):")
    print(df_cleaned[sample_cols].head(3).to_string(max_colwidth=50))

    # Opslaan
    df_cleaned.to_csv(output_file, index=False)
    print(f"\n{'=' * 60}")
    print(f"OPGESLAGEN: {output_file}")
    print(f"{'=' * 60}")

    return df_cleaned


if __name__ == "__main__":
    df_cleaned = clean_vkm_dataset(INPUT_FILE, OUTPUT_FILE)


DATA CLEANING & ENRICHMENT PROCESS
Origineel: 211 rijen, 20 kolommen

1. Kolommen geselecteerd: 16 van 16
   Behouden: ['id', 'name', 'shortdescription', 'description', 'content', 'studycredit', 'location', 'contact_id', 'level', 'learningoutcomes', 'module_tags', 'interests_match_score', 'popularity_score', 'estimated_difficulty', 'available_spots', 'start_date']

2. Vervangen van 'ntb' waarden
   shortdescription: 10 'ntb' waarden vervangen
   description: 2 'ntb' waarden vervangen
   content: 2 'ntb' waarden vervangen
   learningoutcomes: 26 'ntb' waarden vervangen

2b. Auto-genereren van module tags voor modules zonder tags
   30 modules zonder tags → 0 na auto-generatie

3. Vullen van lege content velden
   shortdescription: 20 lege velden → 0 na vullen
   description: 0 lege velden → 0 na vullen
   content: 0 lege velden → 0 na vullen

4. Genereren van learning outcomes
   36 ontbrekende learning outcomes → 0 na vullen

5. start_date geconverteerd naar datetime
   Ongeldige datum

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
